# Sorting DataFrames

## Install packages

In [ ]:
!pip install advertools adviz matplotlib -q

## Import packages:

In [ ]:
import advertools as adv
import adviz
import pandas as pd
import random
random.seed(2)

## Creating sample data (ignore)

In [1]:
products = []
prod = ["Toaster", "Blender", "Microwave", "Cooker", "CoffeeMaker"]
for i in range(3):
    random.shuffle(prod)
    products.extend(prod)


sales = [random.expovariate(.1) for i in range(15)]

df = pd.DataFrame({
    "date": [10 for i in range(5)] + [11 for i in range(5)] + [12 for i in range(5)],
    "day": ['Monday' for i in range(5)] + ['Tuesday' for i in range(5)] + ['Wednesday' for i in range(5)],
    "product": products,
    "sales": sales
})
df['sales'] = df['sales'].round(0)
df['product'] = df['product'].astype('category')
df.style.set_caption("<h3>Sales report")

,date,day,product,sales
0,10,Monday,Microwave,82.000000
1,10,Monday,Blender,10.000000
2,10,Monday,Cooker,16.000000
3,10,Monday,CoffeeMaker,20.000000
4,10,Monday,Toaster,7.000000
5,11,Tuesday,CoffeeMaker,5.000000
6,11,Tuesday,Microwave,27.000000
7,11,Tuesday,Toaster,7.000000
8,11,Tuesday,Cooker,23.000000
9,11,Tuesday,Blender,20.000000


## Have you sorted a table using one of its columns?

In [11]:
adviz.style_table(df
 .sort_values(["sales"],
              ascending=[False]), column_types=['text', 'text', 'text', 'bar'],
                  width=750,
                  title="Sorted by sales"
 )

In [12]:
adviz.style_table(df
 .sort_values(["date"],
              ascending=[False]), column_types=['category', 'text', 'text', 'text'],
                  width=750,
                  title="Sorted by date"
 )

## Have you sorted a table using two of its columns?

- Sort by date, then by sales
- Get the sales per day, sorted for each day

In [17]:
adviz.style_table(df
 .sort_values(["date", "sales"],
              ascending=[True, False]), column_types=['text', 'category', 'text', 'bar'],
                  width=750,
                  title="Sorted by date, then by sales<br>Top selling products for each day"
)

- Sales by product so you know which days sold more

In [18]:
adviz.style_table(df
 .sort_values(by=["product", "sales"],
              ascending=[True, False]),
                  column_types=['text', 'text', 'category', 'bar'],
                  width=750,
                  title="Sorted by product, then by sales<br>Top selling days per product"
)

## Website traffic
- Get the top visited pages by country
- Still too many columns
- Unreadable

In [21]:
local_url = "ga_data.csv"
github_url = "https://raw.githubusercontent.com/eliasdabbas/foundconf/refs/heads/main/ga_data.csv"


try:
    ga = pd.read_csv(local_url)
except FileNotFoundError:
    ag = pd.read_csv(github_url)

ga = ga.rename(columns={
    "Page path and screen class": 'Path',
    "View per active user": "Views",
    "Average engagement time per active user": "Engagement time"})
ga = ga[~ga['Path'].isin(["/about", "/about.html"])]
ga['Path'] = ga['Path'].str.removesuffix("/") + "/"
ga

,Path,Country,Views,Active users,Views per active user,Engagement time,Event count,Key events,Total revenue
0,/,China,116,115,1.008696,1.904348,424,0,0
1,/xml-sitemaps/,China,92,21,4.380952,109.238095,308,0,0
2,/,United States,86,65,1.323077,11.615385,345,0,0
3,/,India,75,39,1.923077,16.461538,289,0,0
4,/,United Kingdom,71,47,1.510638,17.872340,291,0,0
...,...,...,...,...,...,...,...,...,...
1093,/urlytics/,Singapore,0,1,0.000000,24.000000,1,0,0
1094,/urlytics/,Singapore,0,2,0.000000,4.000000,3,0,0
1095,/user-agent-parser/,Singapore,0,3,0.000000,4.000000,5,0,0
1096,/xml-sitemaps/,Singapore,0,1,0.000000,5.000000,1,0,0


In [7]:
ga.sort_values(['Path', "Views"], ascending=[True, False])

,Path,Country,Views,Active users,Views per active user,Engagement time,Event count,Key events,Total revenue
0,/,China,116,115,1.008696,1.904348,424,0,0
2,/,United States,86,65,1.323077,11.615385,345,0,0
3,/,India,75,39,1.923077,16.461538,289,0,0
4,/,United Kingdom,71,47,1.510638,17.872340,291,0,0
8,/,Türkiye,43,12,3.583333,19.916667,122,0,0
...,...,...,...,...,...,...,...,...,...
1039,/xml-sitemaps/,Saudi Arabia,1,1,1.000000,19.000000,4,0,0
1040,/xml-sitemaps/,South Korea,1,1,1.000000,0.000000,6,0,0
1041,/xml-sitemaps/,Sri Lanka,1,1,1.000000,5.000000,3,0,0
1096,/xml-sitemaps/,Singapore,0,1,0.000000,5.000000,1,0,0


## Top `n` rows per group*

- group*: Any set of rows you want (`Path` in this case)

In [20]:
(ga
    .sort_values(['Path', "Event count"],
                 ascending=[True, False])
    .groupby("Path")
    .head()
    .head(30)[['Path', "Country", "Event count"]]
    .style
    .bar(subset=["Event count"], color="steelblue")
    .set_caption("<h4>Top countries per page"))

,Path,Country,Event count
0,/,China,424
2,/,United States,345
4,/,United Kingdom,291
3,/,India,289
8,/,Türkiye,122
64,/about/,Croatia,33
129,/about/,United States,17
153,/about/,Switzerland,15
188,/about/,India,13
345,/about/,Vietnam,7


## Get top countries by `Event count` (or any metric)

In [9]:
top_countries = ga.groupby("Country", as_index=False).sum("Event count").sort_values("Event count", ascending=False).head(10)
top_countries

,Country,Views,Active users,Views per active user,Engagement time,Event count,Key events,Total revenue
87,United States,455,342,80.815620,1938.608979,1671,0,0
17,China,387,313,78.889648,187.059110,1364,0,0
35,India,273,199,58.168298,1673.477843,1009,0,0
86,United Kingdom,265,205,52.263283,1276.153539,946,0,0
31,Germany,160,116,63.769774,1672.309050,577,0,0
79,Switzerland,118,62,46.891667,1361.683333,380,0,0
40,Italy,114,55,46.892857,2117.450000,377,0,0
83,Türkiye,117,55,45.533333,1508.645238,365,0,0
60,Pakistan,98,47,25.256277,513.282828,325,0,0
15,Canada,87,62,33.152778,703.619444,325,0,0


## Get the top pages per country (from the top countries)

In [10]:
(ga[ga['Country'].isin(top_countries['Country'][:6])]
    .sort_values(['Country', "Event count"],
                 ascending=[True, False])
    .groupby("Country")
    .head()
    .head(30)[["Country", 'Path', "Event count"]]
    .style
    .bar(subset=["Event count"], color="steelblue"))

,Country,Path,Event count
0,China,/,424
1,China,/xml-sitemaps/,308
51,China,/entity-extraction/,42
65,China,/audience-manager/,36
143,China,/seo-crawler/,25
12,Germany,/python/tutorial/running-python-scripts-uv/,108
13,Germany,/,105
30,Germany,/seo-crawler/,78
61,Germany,/xml-sitemaps/,50
81,Germany,/reverse-dns-lookup/,32
